In [90]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import AlphaNet_LSTM_V1
# read_task
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()

# device = task_info["Cuda"].pop()
# task = task_info["Task"]
# task_index = task[task["status"] == "waiting"].index[0]
# Alpha_Name = task.loc[task_index,"Alpha_Name"]
# start_date = task.loc[task_index,"start_date"]
# end_date = task.loc[task_index,"end_date"]
# sequence = task.loc[task_index,"sequence"]
# LR = task.loc[task_index,"LR"]
# epoch_num = task.loc[task_index,"epoch_num"]
# feature_num = task.loc[task_index,"feature_num"]
# task.loc[task_index,"status"] = device
# np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)

In [91]:
task_info["Factor"]

,Alpha_Name,shift,sequence,target,LR,epoch_num,alpha_list,status
factor_id,,,,,,,,
0,AlphaNet_ConcatMoving5And10,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_5, COV_open_high_10, COV_open_l...",finished
1,AlphaNet_No_Decay,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
2,AlphaNet_No_Return,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
3,AlphaNet_No_ZSCORE,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
4,AlphaNet_No_STD,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
5,AlphaNet_No_CORR,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
6,AlphaNet_No_COV,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[CORR_open_high_10, CORR_open_low_10, CORR_ope...",waiting


In [92]:
alpha_name = "AlphaNetV1"
shift = 1
sequence = 20
target = "10d_ret_standscalar"
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]

In [93]:
alpha_list = [
'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["CORR","STD","ZSCORE","RETURN","DECAY","COV"
         ]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_open_high_10',
 'COV_open_low_10',
 'COV_open_close_10',
 'COV_open_vwap_10',
 'COV_open_volume_10',
 'COV_open_pct_chg_10',
 'COV_open_turnover_10',
 'COV_open_free_turnover_10',
 'COV_high_low_10',
 'COV_high_close_10',
 'COV_high_vwap_10',
 'COV_high_volume_10',
 'COV_high_pct_chg_10',
 'COV_high_turnover_10',
 'COV_high_free_turnover_10',
 'COV_low_close_10',
 'COV_low_vwap_10',
 'COV_low_volume_10',
 'COV_low_pct_chg_10',
 'COV_low_turnover_10',
 'COV_low_free_turnover_10',
 'COV_close_vwap_10',
 'COV_close_volume_10',
 'COV_close_pct_chg_10',
 'COV_close_turnover_10',
 'COV_close_free_turnover_10',
 'COV_vwap_volume_10',
 'COV_vwap_pct_chg_10',
 'COV_vwap_turnover_10',
 'COV_vwap_free_turnover_10',
 'COV_volume_pct_chg_10',
 'COV_volume_turnover_10',
 'COV_volume_free_turnover_10',
 'COV_pct_chg_turnover_10',
 'COV_pct_chg_free_turnover_10',
 'COV_turnover_free_turnover_10',
 'CORR_open_high_10',
 'CORR_open_low_10',
 'CORR_open_close_10',
 'CORR_open_vwap_10',
 'CORR_open_

In [94]:
factor_df = pd.DataFrame([alpha_name,shift,sequence,target,LR,epoch_num,alpha_list],index=[
    'Alpha_Name','shift', 'sequence','target', 'LR', 'epoch_num' ,'alpha_list'
]).T
factor_df.index.names = ["factor_id"]
factor_df["status"] = "waiting"

In [95]:
task_info["Factor"] = task_info["Factor"].append(factor_df)
task_info["Factor"].reset_index(inplace=True,drop=True)
task_info["Factor"].index.names = ["factor_id"]
task_info["Factor"]

,Alpha_Name,shift,sequence,target,LR,epoch_num,alpha_list,status
factor_id,,,,,,,,
0,AlphaNet_ConcatMoving5And10,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_5, COV_open_high_10, COV_open_l...",finished
1,AlphaNet_No_Decay,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
2,AlphaNet_No_Return,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
3,AlphaNet_No_ZSCORE,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
4,AlphaNet_No_STD,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
5,AlphaNet_No_CORR,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting
6,AlphaNet_No_COV,5,6,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[CORR_open_high_10, CORR_open_low_10, CORR_ope...",waiting
7,AlphaNetV1,1,20,10d_ret_standscalar,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_10, COV_open_low_10, COV_open_c...",waiting


In [96]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)